In [ ]:
import pandas as pd
#TODO Defined correct path
CHEMIN_PROCCESSED_DATA_FOLDER = "../processed/"

# 1. Charger le fichier parquet
#TODO utilise la fonction ingestion d'Emna et Shams
df = pd.read_parquet(CHEMIN_PROCCESSED_DATA_FOLDER+"processed_data.parquet")
df



In [ ]:
#Split en test et train
from sklearn.model_selection import train_test_split

# Diviser en train et test (par exemple 80/20)
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, LGBMRanker
from catboost import CatBoostClassifier

# -------------------------------------------------------------------
# Pourquoi tester ces algorithmes pour la recommandation de produits
# en cas de rupture de stock ?
#
# L’objectif est de prédire ou de classer les meilleurs produits de
# substitution à proposer à un client lorsqu’un article est indisponible.
# Nous testons plusieurs familles de modèles afin de comparer :
#   - la capacité de généralisation,
#   - la performance en classification et en ranking,
#   - la robustesse face à des features hétérogènes (prix, catégorie,
#     similarité, contexte client, etc.).
#
# 1) Logistic Regression
#    - Sert de modèle baseline simple et interprétable.
#    - Permet de vérifier que les features contiennent bien un signal
#      prédictif (sanity check).
#    - Facilite l’analyse des coefficients et la compréhension métier.
#
# 2) XGBClassifier (XGBoost)
#    - Modèle de gradient boosting très performant sur données tabulaires.
#    - Capable de capturer des relations non linéaires complexes entre
#      produits et contexte client.
#    - Sert de référence "haut niveau" en classification supervisée.
#
# 3) LGBMClassifier (LightGBM)
#    - Alternative plus rapide et scalable au XGBoost.
#    - Très efficace sur de grands volumes de données et de nombreuses
#      features.
#    - Utilisé ici pour prédire la probabilité qu’un produit substitut
#      soit accepté par le client.
#
# 4) LGBMRanker (LightGBM - Learning to Rank)
#    - Modèle spécifiquement conçu pour les problèmes de ranking.
#    - Permet de classer plusieurs produits candidats pour une même
#      rupture de stock et de sélectionner le meilleur (Top-1 ou Top-K).
#    - Particulièrement adapté aux systèmes de recommandation.
#
# 5) CatBoostClassifier
#    - Modèle de boosting optimisé pour les variables catégorielles.
#    - Réduit le besoin de preprocessing (encodage) des catégories.
#    - Souvent très performant dans les contextes e-commerce avec
#      catégories, marques et attributs produits.
#
# Cette approche multi-modèles permet d’identifier le meilleur compromis
# entre performance, interprétabilité et robustesse pour le moteur de
# recommandation de produits de substitution.
# -------------------------------------------------------------------


# Définir les modèles et leurs grilles de paramètres
models = {

    # =========================
    # Logistic Regression (Baseline)
    # =========================
    "LogReg": {
        "model_class": LogisticRegression,
        "param_grid": {
            "C": [0.1, 1.0, 10.0],
            "penalty": ["l2"],
        },
        "fixed_params": {
            "solver": "lbfgs",
            "max_iter": 2000,
            "n_jobs": -1,
            "random_state": 42,
        },
    },

    # =========================
    # XGBoost Classifier
    # =========================
    "XGBClassifier": {
        "model_class": XGBClassifier,
        "param_grid": {
            "n_estimators": [500, 1000],
            "max_depth": [4, 6, 8],
            "learning_rate": [0.03, 0.05, 0.1],
            "subsample": [0.7, 0.9, 1.0],
            "colsample_bytree": [0.7, 0.9, 1.0],
            "min_child_weight": [1, 5, 10],
            "reg_alpha": [0.0, 0.1, 1.0],
            "reg_lambda": [1.0, 2.0, 5.0],
        },
        "fixed_params": {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "tree_method": "hist",
            "random_state": 42,
            "n_jobs": -1,
        },
    },

    # =========================
    # LightGBM Classifier
    # =========================
    "LGBMClassifier": {
        "model_class": LGBMClassifier,
        "param_grid": {
            "num_leaves": [31, 63, 127],
            "learning_rate": [0.03, 0.05, 0.1],
            "n_estimators": [500, 1000],
            "min_child_samples": [20, 50, 100],
            "subsample": [0.7, 0.9, 1.0],
            "colsample_bytree": [0.7, 0.9, 1.0],
            "reg_alpha": [0.0, 0.1, 1.0],
            "reg_lambda": [0.0, 0.1, 1.0],
        },
        "fixed_params": {
            "objective": "binary",
            "metric": "auc",
            "random_state": 42,
            "n_jobs": -1,
        },
    },

    # =========================
    # LightGBM Ranker
    # =========================
    "LGBMRanker": {
        "model_class": LGBMRanker,
        "param_grid": {
            "objective": ["lambdarank"],
            "metric": ["ndcg"],
            "num_leaves": [31, 63, 127],
            "learning_rate": [0.03, 0.05, 0.1],
            "n_estimators": [500, 1000],
            "min_child_samples": [20, 50, 100],
            "subsample": [0.7, 0.9, 1.0],
            "colsample_bytree": [0.7, 0.9, 1.0],
        },
        "fixed_params": {
            "random_state": 42,
            "n_jobs": -1,
        },
    },

    # =========================
    # CatBoost Classifier
    # =========================
    "CatBoostClassifier": {
        "model_class": CatBoostClassifier,
        "param_grid": {
            "depth": [6, 8, 10],
            "learning_rate": [0.03, 0.05, 0.1],
            "iterations": [500, 1000],
            "l2_leaf_reg": [1, 3, 5, 9],
            "subsample": [0.7, 0.9, 1.0],
            "rsm": [0.7, 0.9, 1.0],
        },
        "fixed_params": {
            "loss_function": "Logloss",
            "eval_metric": "AUC",
            "random_seed": 42,
            "verbose": 0,
        },
    },
}



In [ ]:
import numpy as np
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss,
    precision_score, recall_score, f1_score
)
# Métriques ranking simples (binaire ou grades) 
def dcg_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    if rels.size == 0:
        return 0.0
    discounts = 1.0 / np.log2(np.arange(2, rels.size + 2))
    return float(np.sum(rels * discounts))

def ndcg_at_k(rels, k):
    dcg = dcg_at_k(rels, k)
    ideal = dcg_at_k(sorted(rels, reverse=True), k)
    return 0.0 if ideal == 0 else float(dcg / ideal)

def mrr_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    hits = np.where(rels > 0)[0]
    return 0.0 if hits.size == 0 else float(1.0 / (hits[0] + 1))

def hit_rate_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    return float(np.any(rels > 0))

def precision_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    return float(np.mean(rels > 0)) if rels.size else 0.0

def recall_at_k(rels, k):
    rels = np.asarray(rels)
    total_pos = np.sum(rels > 0)
    if total_pos == 0:
        return 0.0
    return float(np.sum(rels[:k] > 0) / total_pos)

In [ ]:
def compute_ranking_metrics(y_true, y_score, group, ks=(1, 3, 5)):
    """
    y_true: array (0/1 ou grades) trié dans le même ordre que y_score, group
    group: liste des tailles de groupes (ex: [20, 20, 15, ...]) -> une requête = un stockout event
    """
    metrics = {}
    idx = 0

    # accumulation moyenne par requête
    per_k = {k: {"ndcg": [], "mrr": [], "hit": [], "p": [], "r": []} for k in ks}

    for g in group:
        y_g = np.asarray(y_true[idx: idx + g])
        s_g = np.asarray(y_score[idx: idx + g])
        idx += g

        order = np.argsort(-s_g)  # desc
        rels_sorted = y_g[order]

        for k in ks:
            per_k[k]["ndcg"].append(ndcg_at_k(rels_sorted, k))
            per_k[k]["mrr"].append(mrr_at_k(rels_sorted, k))
            per_k[k]["hit"].append(hit_rate_at_k(rels_sorted, k))
            per_k[k]["p"].append(precision_at_k(rels_sorted, k))
            per_k[k]["r"].append(recall_at_k(rels_sorted, k))
    for k in ks:
        metrics[f"ndcg@{k}"] = float(np.mean(per_k[k]["ndcg"]))
        metrics[f"mrr@{k}"] = float(np.mean(per_k[k]["mrr"]))
        metrics[f"hit_rate@{k}"] = float(np.mean(per_k[k]["hit"]))
        metrics[f"precision@{k}"] = float(np.mean(per_k[k]["p"]))
        metrics[f"recall@{k}"] = float(np.mean(per_k[k]["r"]))

    return metrics

In [ ]:
# TODO Definir les groups train pour les algo de ranking

In [ ]:
import mlflow

# Initialiser MlFlow
mlflow.set_tracking_uri("http://localhost:5555")
client = mlflow.MlflowClient()
mlflow.set_experiment("Algo_recommandation")

In [ ]:
from sklearn.model_selection import ParameterGrid

# Boucle MLflow
for model_name, config in models.items():
    model_class = config["model_class"]
    param_grid = config["param_grid"]
    fixed_params = config["fixed_params"]

    for params in ParameterGrid(param_grid):
        full_params = {**params, **fixed_params}

        with mlflow.start_run():
            mlflow.set_tags({
                "model": model_name,
                "experiment": "stockout_substitution_reco_comparison",
                "task": "ranking" if model_name == "LGBMRanker" else "binary_classification",
            })

            mlflow.log_params(full_params)

            try:
                model = model_class(**full_params)

                if model_name == "LGBMRanker":
                    # IMPORTANT: LightGBM Ranker nécessite group_train/group_val (ta granularité = stockout event)
                    # y_train = pertinence du candidat substitut (0/1 ou grades)
                    model.fit(
                        X_train, y_train,
                        group=group_train,
                        eval_set=[(X_val, y_val)],
                        eval_group=[group_val],
                    )

                    val_scores = model.predict(X_val)
                    rank_metrics = compute_ranking_metrics(y_val, val_scores, group_val, ks=(1, 3, 5))
                    mlflow.log_metrics(rank_metrics)

                    # Logger le modèle (MLflow LightGBM si dispo, sinon sklearn)
                    try:
                        import mlflow.lightgbm
                        mlflow.lightgbm.log_model(model, "model")
                    except Exception:
                        mlflow.sklearn.log_model(model, "model")

                else:
                    # CLASSIFICATION
                    # (Si tu as un déséquilibre, pense à pondérer: class_weight='balanced' / scale_pos_weight pour XGB)
                    model.fit(X_train, y_train)

                    # proba pour métriques probabilistes
                    if hasattr(model, "predict_proba"):
                        proba = model.predict_proba(X_val)[:, 1]
                    else:
                        # fallback (rare)
                        proba = model.predict(X_val)

                    pred = (proba >= 0.5).astype(int)

                    metrics = {
                        "auc": float(roc_auc_score(y_val, proba)) if len(np.unique(y_val)) > 1 else np.nan,
                        "pr_auc": float(average_precision_score(y_val, proba)) if len(np.unique(y_val)) > 1 else np.nan,
                        "logloss": float(log_loss(y_val, proba, eps=1e-15)),
                        "precision": float(precision_score(y_val, pred, zero_division=0)),
                        "recall": float(recall_score(y_val, pred, zero_division=0)),
                        "f1": float(f1_score(y_val, pred, zero_division=0)),
                    }

                    # BONUS: métriques de ranking même pour les classif (on score et on trie)
                    # Nécessite group_val pour pouvoir calculer topK par stockout event.
                    if "group_val" in globals() and group_val is not None:
                        metrics.update(compute_ranking_metrics(y_val, proba, group_val, ks=(1, 3, 5)))

                    mlflow.log_metrics(metrics)

                    # Logger le modèle selon lib
                    if model_name == "XGBClassifier":
                        try:
                            import mlflow.xgboost
                            mlflow.xgboost.log_model(model, "model")
                        except Exception:
                            mlflow.sklearn.log_model(model, "model")
                    elif model_name in ["LGBMClassifier"]:
                        try:
                            import mlflow.lightgbm
                            mlflow.lightgbm.log_model(model, "model")
                        except Exception:
                            mlflow.sklearn.log_model(model, "model")
                    elif model_name == "CatBoostClassifier":
                        try:
                            import mlflow.catboost
                            mlflow.catboost.log_model(model, "model")
                        except Exception:
                            mlflow.sklearn.log_model(model, "model")
                    else:
                        mlflow.sklearn.log_model(model, "model")

            except Exception as e:
                mlflow.log_param("error", str(e))
                raise